In [8]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import sys
from scipy import misc


def get_filenames(img_dir, class_idx):
    imgs = []
    img_idx = 0
    image_list = glob.glob(os.path.join(img_dir, "*.png"))
    image_list.sort()
    for img in image_list:
        if os.path.isfile(img):
            fsplit = img.split('_')
            if int(fsplit[-2]) == class_idx:
                imgs.append(img)

    return imgs

def img_to_rates(img, middle_level=127., min_rate=5., max_rate=50.):
    _on  = np.where(img > middle_level)
    _off = np.where(img <= middle_level)
    np.random.seed()
    rates_on = np.random.randint(0, min_rate+1, size=img.shape).astype('float32')
    rates_on[_off] = 0.

    np.random.seed()
    rates_off = np.random.randint(0, min_rate+1, size=img.shape).astype('float32')
    rates_off[_on] = 0. 
    
    rates_on[_on]   += np.round( ((img[_on] - middle_level) * max_rate)/(middle_level) )
    rates_off[_off] += np.round( ((middle_level - img[_off] ) * max_rate)/(middle_level) )
    rates_on[:] = np.clip(rates_on, 0, max_rate+min_rate)
    rates_off[:] = np.clip(rates_off, 0, max_rate+min_rate)
    return rates_on, rates_off

def process_all(in_path, setname, class_idx, out_path, width, height, kernel=None,
                middle_level=np.round(255*0.27), min_rate=5., max_rate=50., n_imgs=-1):
    img_paths = get_filenames(os.path.join(in_path, setname), class_idx)
    out_template = 'class_%d__%06d_samples__width_%03d__height_%03d__%s_channel__midLevel_%03d__maxRate_%02d.npy'
    file_on  = out_template%(class_idx, len(img_paths), width, height, 'on', middle_level, max_rate)
    file_off = out_template%(class_idx, len(img_paths), width, height, 'off', middle_level, max_rate)
    
    arr_on  = np.memmap(os.path.join(out_path, file_on), dtype='float32', 
                mode='w+', shape=(len(img_paths), height, width))
    arr_off = np.memmap(os.path.join(out_path, file_off), dtype='float32', 
                mode='w+', shape=(len(img_paths), height, width))
    img = np.zeros((height, width))
    
    for i, pth in enumerate(img_paths[:n_imgs]):
        img[:] = misc.imread(pth, flatten=True)
        if kernel is not None:
            ### convolve the image with the kernel and then separate
            pass
        arr_on[i,:,:], arr_off[i, :, :] = img_to_rates(img, middle_level, min_rate, max_rate)

#     print(arr_on.shape)


setname = 'train'
class_idx = 0
img_w, img_h = 28, 28
img_path = './'
out_path = './'
for cls in range(0, 10):
    print("processing class %d"%cls)
    process_all(img_path, setname, cls, out_path, img_w, img_h)


processing class 0


IOError: [Errno 22] Invalid argument

In [ ]:
# plt.figure()
# ax = plt.subplot(1, 3, 1)
# plt.imshow(img, interpolation='none')

# ax = plt.subplot(1, 3, 2)
# plt.imshow(rates_on, interpolation='none')

# ax = plt.subplot(1, 3, 3)
# plt.imshow(rates_off, interpolation='none')

# plt.show()